# Mixture of Gaussians

In this note, we will solve the mixture of Gaussians

This block contains some $\LaTeX$ macros.
$\newcommand{\E}{\mathbb{E}}$
$\renewcommand{\Re}{\mathbb{R}}$
$\newcommand{\oft}[1]{{^{(#1)}}}$
$\newcommand{\oftt}[2]{{^{(#1)#2}}}$

In [49]:
import util
from util import *
import numpy as np
import models
import mompy as mp
import cvxopt

np.set_printoptions(precision=3, suppress=True)
from IPython.display import display, Markdown, Math
from operator import mul
import sympy as sp

sp.init_printing()

## Noiseless Example

Let's construct a toy example for the rest of this document.

Let $\xi_1, ..., \xi_K \in \Re^D$ be the set of means. Let $c_1, ..., c_K \in \Re^D$ be the set of diagonal covariances.

In [50]:
k = 3
d = 3
degobs = 3
degmm = 2

sc.random.seed(101)
gm = models.GaussianMixtureModel.generate('', k, d, means='random', cov='spherical')

constraints = gm.polymom_all_constraints(degobs)
xis = gm.sym_means
covs = gm.sym_covs
sym_all = xis + covs
print 'The polynomial constraints are as follows'
display(constraints)

The polynomial constraints are as follows


With this machinery, we can compute the moment polynomials required for the moment method magic!

In [51]:
MM = mp.MomentMatrix(degmm, sym_all, morder='grevlex')
display(MM.row_monos)

In [52]:
cin = mp.solvers.get_cvxopt_inputs(MM, constraints)
solsdp = cvxopt.solvers.sdp(cin['c'], Gs=cin['G'], hs=cin['h'], A=cin['A'], b=cin['b'])


     pcost       dcost       gap    pres   dres   k/t
 0:  2.8905e+00  2.8905e+00  2e+02  2e+01  1e+00  1e+00
 1:  2.8888e+01  3.2162e+01  3e+02  2e+01  1e+00  4e+00
 2:  1.7049e+01  1.9702e+01  2e+01  2e+00  1e-01  3e+00
 3:  2.0962e+01  2.2383e+01  1e+01  1e+00  8e-02  1e+00
 4:  2.1680e+01  2.2087e+01  4e+00  4e-01  2e-02  4e-01
 5:  2.2302e+01  2.2768e+01  5e+00  5e-01  3e-02  5e-01
 6:  2.2726e+01  2.2901e+01  2e+00  2e-01  1e-02  2e-01
 7:  2.3590e+01  2.3788e+01  4e+00  2e-01  1e-02  2e-01
 8:  2.3947e+01  2.3992e+01  1e+00  8e-02  4e-03  5e-02
 9:  2.4141e+01  2.4162e+01  7e-01  3e-02  2e-03  2e-02
10:  2.4225e+01  2.4230e+01  2e-01  9e-03  5e-04  6e-03
11:  2.4254e+01  2.4256e+01  7e-02  3e-03  2e-04  2e-03
12:  2.4263e+01  2.4263e+01  1e-02  7e-04  4e-05  5e-04
13:  2.4265e+01  2.4266e+01  4e-03  2e-04  1e-05  1e-04
14:  2.4266e+01  2.4266e+01  7e-04  3e-05  2e-06  2e-05
15:  2.4266e+01  2.4266e+01  2e-04  1e-05  6e-07  8e-06
16:  2.4266e+01  2.4266e+01  4e-05  2e-06  1e-07  

In [53]:
sol_lasserre = mp.extractors.extract_solutions_lasserre(MM, solsdp['x'], Kmax = k)
sol_dreesen = mp.extractors.extract_solutions_dreesen_proto(MM, solsdp['x'], Kmax = k)
trueparams = {}

for i in xrange(d):
    trueparams[xis[i]] = []
    trueparams[covs[i]] = []
    for j in xrange(k):
        trueparams[xis[i]].append(gm.means[i,j])
        trueparams[covs[i]].append(gm.sigmas[j,i,i])
print 'the true parameters'
display(trueparams)
print 'recovered parameters (Lasserre)'
display(sol_lasserre)
print 'recovered parameters (Dreesen)'
display(sol_dreesen)

the biggest singular value we are losing is 0.453184
the next biggest eigenvalue we are losing is 0.453184
the true parameters


recovered parameters (Lasserre)


recovered parameters (Dreesen)


## Now take samples!


In [54]:
num_samples = 10]]

0
X = gm.sample(num_samples)
import matplotlib.pyplot as plt
%matplotlib inline

import numpy.linalg

# plot after a random orthogonal projection
randdir,__,__ = np.linalg.svd(np.random.randn(d,2))
Y = X.dot(randdir[:,0:2])
plt.scatter(Y[:,0], Y[:,1]);

constraints_noisy = gm.polymom_all_constraints_samples(3, X)
display(constraints_noisy)

SyntaxError: invalid syntax (<ipython-input-54-1f9d6863b1ee>, line 1)

In [35]:
cin = mp.solvers.get_cvxopt_inputs(MM, constraints_noisy)
solsdp_noisy = cvxopt.solvers.sdp(cin['c'], Gs=cin['G'], hs=cin['h'], A=cin['A'], b=cin['b'])


     pcost       dcost       gap    pres   dres   k/t
 0:  1.4287e+00  1.4287e+00  7e+01  7e+00  1e+00  1e+00
 1:  1.3875e+00  1.9303e+00  6e+00  1e+00  2e-01  7e-01
 2:  2.7793e+00  3.2285e+00  8e+00  1e+00  2e-01  6e-01
 3:  3.0147e+00  3.1027e+00  1e+00  2e-01  4e-02  1e-01
 4:  3.0799e+00  3.1297e+00  9e-01  1e-01  2e-02  7e-02
 5:  3.1189e+00  3.1274e+00  2e-01  2e-02  5e-03  1e-02
 6:  3.1315e+00  3.1330e+00  4e-02  6e-03  1e-03  2e-03
 7:  3.1328e+00  3.1330e+00  7e-03  9e-04  2e-04  4e-04
 8:  3.1335e+00  3.1336e+00  3e-03  4e-04  8e-05  2e-04
 9:  3.1338e+00  3.1338e+00  6e-04  7e-05  1e-05  3e-05
10:  3.1338e+00  3.1338e+00  4e-04  5e-05  9e-06  2e-05
11:  3.1339e+00  3.1339e+00  2e-05  2e-06  5e-07  9e-07
12:  3.1339e+00  3.1339e+00  1e-06  2e-07  3e-08  5e-08
13:  3.1339e+00  3.1339e+00  4e-08  5e-09  1e-09  2e-09
Optimal solution found.


In [36]:
sol_lasserre_noisy = mp.extractors.extract_solutions_lasserre(MM, solsdp_noisy['x'], Kmax = k)
sol_dreesen_noisy = mp.extractors.extract_solutions_dreesen_proto(MM, solsdp_noisy['x'], Kmax = k)

print 'the true parameters'
display(trueparams)
print 'recovered parameters (Lasserre)'
display(sol_lasserre_noisy)
print 'recovered parameters (Dreesen)'
display(sol_dreesen_noisy)

the biggest singular value we are losing is 0.000626
the next biggest eigenvalue we are losing is 0.000626
the true parameters


recovered parameters (Lasserre)


recovered parameters (Dreesen)


## EM Algorithm

In [37]:
from algos import GaussianMixturesEM
reload(GaussianMixturesEM)
algo = GaussianMixturesEM.GaussianMixtureEM( k, d )
X.shape
lhood, Z, O_ = algo.run( X, None )

Iteration 0, lhood = 4.808392
Iteration 1, lhood = 6.154847
Iteration 2, lhood = 6.160424
Iteration 3, lhood = 6.160424
Iteration 4, lhood = 6.160424
Iteration 5, lhood = 6.160424
Iteration 6, lhood = 6.160424
Iteration 7, lhood = 6.160424
Iteration 8, lhood = 6.160424
Iteration 9, lhood = 6.160424
Iteration 10, lhood = 6.160424
Iteration 11, lhood = 6.160424
Converged with lhood=6.160424 in 11 steps.


In [38]:
O_, gm.weights

((array([[ 1.012, -0.009,  0.018],
         [ 0.008, -0.021,  0.934],
         [ 0.012,  0.993,  0.044]]),
  0.095604762696739468,
  array([ 0.515,  0.311,  0.175])),
 array([ 0.458,  0.26 ,  0.282]))

In [39]:
gm.sigmas

array([[[ 0.01,  0.  ,  0.  ],
        [ 0.  ,  0.01,  0.  ],
        [ 0.  ,  0.  ,  0.01]],

       [[ 0.01,  0.  ,  0.  ],
        [ 0.  ,  0.01,  0.  ],
        [ 0.  ,  0.  ,  0.01]],

       [[ 0.01,  0.  ,  0.  ],
        [ 0.  ,  0.01,  0.  ],
        [ 0.  ,  0.  ,  0.01]]])